In [1]:
from __future__ import division

import config
import os, numpy as np, pandas, sklearn, scipy.signal as signal
import mido
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import bz2 

In [3]:
# local libs
import config, models, setup, compression
import midi
import midi.decode
from midi import generators as g
from utils import io, models_io, utils, plot
from capsule.layers import Capsule, Length
from capsule.capsulefunctions import squash, softmax, margin_loss

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
context = setup.init(max_bars=4)

 Context :: namedtuple(
[ max_t = float
, dt = float
, n_timestesp = int
, note_length = int
, bpm = float
, tempo = float
, ticks_per_beat = int
]

Setting up params

max min f 20.0 0.25
 >> Context(max_t=4.0, dt=0.025, n_timesteps=160, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
 sample length:  160.000000
 max_f: 20.000000, min_f: 0.250000


In [5]:
n = 500*1
dim4 = True
multiTrack = True
reduce_dims = midi.ReduceDimsOptions.MIDIFILE # GLOBAL
dn = 'drum_midi/'
v = None # float | None
x_train, labels = setup.import_data(context, n, dim4=dim4, reduce_dims=reduce_dims, 
    dirname=dn, multiTrack=multiTrack, velocity=v, r=True)


Importing midi-data
 [INFO] : 
 |  path ignored: name `Cha cha` in IGNORE_DIRS
path ignored: ../datasets/drum_midi//Cha cha 

path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/05 Straight Line 136BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/05 Straight Line 136BPM
path ../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/05 Straight Line 136BP


Encoding midi-data
 500
> -> multi-track = True MidiFile
 [INFO] : 
 |  True


In [6]:
[(k,v) for k,v in enumerate(labels)]

[(0,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/05 8th Hat.mid"),
 (1,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/03 8th Ride.mid"),
 (2,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/04 8th Perc.mid"),
 (3,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/06 8th Perc.mid"),
 (4,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/02 8th Hat.mid"),
 (5,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/07 8th Perc.mid"),
 (6,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/06 Moonglow 140BPM/01 8th Hat.mid"),
 (7,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/05 Straight Line 136BPM/06 8th hat.mid"),
 (8,
  "../datasets/drum_midi//50´s Drummer MIDI Files/01 Rock'n'Roll/05 Straight Line 136BPM/02 8th Ride.mid"),
 (9,
  "../datasets/dr

In [7]:
io.reset_tmp_dir()

True

In [8]:
x_train.shape

(500, 160, 7, 1)

In [28]:
separator = '/'

In [29]:
len(compression.to_string(x_train[0])), compression.to_string(x_train[0])[:50]

(8479, '[[0.8189,0.0000,0.0000,0.7795,0.0000,0.0000,0.0000')

In [30]:
compression.K(x_train[0])

265

In [31]:
compression.NCD_multiple(x_train[0:10], x_train[10:20], separator=separator)

0.7820437093916125

In [32]:
compression.NCD_multiple(x_train[0:10], x_train[5:15], pre_concatenation=False, separator=separator)

0.5338036449147561

In [33]:
compression.NCD_multiple(x_train[0:100], x_train[100:200], pre_concatenation=False, separator=separator)

0.9945841633466136

In [34]:
compression.NCD_multiple(x_train[0:100], x_train[50:150], pre_concatenation=False, separator=separator)

0.9972609561752988

In [38]:
compression.NCD_multiple(x_train[0:300], x_train[10:310], pre_concatenation=False, separator=separator)

0.9970232399906445

In [37]:
compression.NCD_multiple(x_train[0:300], x_train[10:310], pre_concatenation=True, separator=separator)

0.9973464583779157

In [ ]:
summed = midi.concatenate(x_train[:4])
compressed = io.compress_MultiTrack(summed[:,:,0])
print('%s: %i' % ('_', len(compressed)))

In [ ]:
summed = midi.concatenate(x_train[[0,10,100,300]])
compressed = io.compress_MultiTrack(summed[:,:,0])
print('%s: %i' % ('_', len(compressed)))

In [ ]:
def K(x):
    k = len(io.compress_MultiTrack(x))
    if k < 0:
        print('Warning: K(x) cannot be <= 0')
    return k

def K_conditional(x,y):
    # K(x|y) = K(xy) - K(x)
    # according to the `symmetry of information` theorem by Li and Vitanyi (1997)
    xy = midi.concatenate([x,y])
    return K(xy) - K(x)

def NCD(x, y, v=0):
    # Normalized Compression Distance
    if x is y:
        if v:
            print('Warning: x is y')
        return 0.
    return max(K_conditional(x,y), K_conditional(y,x)) / max(K(x), K(y))

def NCD_multiple(xs, ys, v=0):
    x = midi.concatenate(xs)
    y = midi.concatenate(ys)
    return NCD(x, y, v)

In [ ]:
NCD_multiple(x_train[0:100], x_train[100:200])

In [ ]:
NCD_multiple(x_train[0:100], x_train[50:150])

In [ ]:
NCD_multiple(x_train[0:10], x_train[10:20])

In [ ]:
NCD_multiple(x_train[0:10], x_train[7:17])

In [ ]:
NCD_multiple(x_train[0:10], x_train[5:15])

In [ ]:
NCD_multiple(x_train[:10], x_train[:10])

In [ ]:
x = midi.concatenate(x_train[:10])
NCD(x, x, v=1)

In [ ]:
def K(x):
    k = len(io.compress_MultiTrack(x))
    if k < 0:
        print('Warning: K(x) cannot be <= 0')
    return k

def K_conditional(x, y, concatenate_post=True):
    # K(x|y) = K(xy) - K(x)
    # according to the `symmetry of information` theorem by Li and Vitanyi (1997)
    if concatenate_post:
        
    else:
        xy = midi.concatenate([x,y])
    return K(xy) - K(x)

In [ ]:
to_string(x_train[0])

In [ ]:
x_train[0].shape

In [ ]:
a = to_string(x).encode('utf-8')
bz2.compress(a)
#         m_ = m.read().encode('utf-8')
#     compressed = bz2.compress(m_)